In [1]:
library(survey)
library(tidyverse)

Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_ph_flag,lab_ph_first,lab_ph_min,lab_ph_max,lab_ph_abnormal,sedative,echo_int,mort_28_day_int,ps,ps_weight
201220,125078,66690,0,2106-04-27 08:47:50,2106-05-01 18:25:46,1,62.67646,1,NA,⋯,1,7.45,7.42,7.45,1,1,0,1,0.4544669,1.833069
215842,151232,11663,0,2188-02-14 08:48:15,2188-02-16 02:02:48,1,86.76186,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.1568838,1.186076
234312,164444,86645,1,2165-06-22 08:47:16,2165-07-07 20:55:20,1,56.08904,1,NA,⋯,1,7.49,7.49,7.49,1,0,0,0,0.3002571,1.429096
289157,146726,10304,0,2156-06-23 21:26:00,2156-06-30 16:26:00,1,45.91093,1,2156-06-25 07:00:00,⋯,1,7.44,7.44,7.44,1,0,1,0,0.6991412,1.430326
211964,160170,94534,0,2160-03-05 21:23:19,2160-03-07 05:48:41,1,59.38693,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.3432285,1.522600
230173,112553,31544,0,2140-01-31 03:39:25,2140-02-03 01:41:39,1,91.50000,1,NA,⋯,0,NA,NA,NA,NA,0,0,1,0.2464983,1.327137


In [4]:
table_one <- data.table::fread(file.path(data_dir, "tableone_raw.csv"), data.table = FALSE)
head(table_one)

p_val,wtd_p_val,main_0,main_1,sub_0,sub_1,wtd_main_0,wtd_main_1,wtd_sub_0,wtd_sub_1,var
4.472893e-02,0.06372199,66.689241,65.824051,17.207039,16.620088,66.847503,66.057657,16.794715,16.639186,age
1.416420e-02,0.34952619,50.822846,47.665687,NA,NA,49.390860,48.199442,NA,NA,female
6.415559e-11,0.12782063,78.562949,82.979049,23.579777,26.702495,80.238931,81.250426,24.310882,25.255760,weight
1.705518e-15,0.07885866,8.505970,10.045054,7.451124,7.681609,9.098576,9.438641,7.547186,7.639513,elix_score
1.378591e-15,0.12030359,19.632462,20.778975,5.788347,5.446293,20.013436,20.232637,5.650021,5.423473,saps
7.217556e-27,0.03089029,5.304937,6.323865,3.618100,3.796365,5.659572,5.861969,3.662696,3.695840,sofa


In [5]:
wpv <- table_one %>%
    filter(wtd_p_val < 0.05) %>%
    pull(var) %>%
    gsub("female", "gender", .)

if (length(grep("day", wpv)) > 0) wpv <- wpv %>% discard(grepl("day", .)) %>% c(., "icu_adm_weekday")

wpv

[1] "sofa"                       "vent"                      
 [3] "vaso"                       "sedative"                  
 [5] "icd_chf"                    "icd_afib"                  
 [7] "lab_chloride_first"         "lab_bnp_flag"              
 [9] "lab_troponin_flag"          "lab_creatinine_kinase_flag"

In [6]:
design_echo_ps_ate <- svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = full_data)

In [7]:
fml <- wpv %>%
c("echo", .) %>%
paste(collapse = " + ") %>%
sprintf("mort_28_day ~ %s", .)

fml

[1] "mort_28_day ~ echo + sofa + vent + vaso + sedative + icd_chf + icd_afib + lab_chloride_first + lab_bnp_flag + lab_troponin_flag + lab_creatinine_kinase_flag"

In [8]:
logi <- svyglm(as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

In [9]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

Survey design:
svydesign(ids = ~icustay_id, weights = ~ps_weight, data = full_data)

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -1.500261   0.494480  -3.034 0.002424 ** 
echo1                       -0.254347   0.071399  -3.562 0.000370 ***
sofa                         0.199117   0.011091  17.953  < 2e-16 ***
vent1                        0.403891   0.107968   3.741 0.000185 ***
vaso1                        0.049657   0.082304   0.603 0.546307    
sedative1                   -0.476685   0.107543  -4.433 9.48e-06 ***
icd_chf1                     0.057116   0.080732   0.707 0.479295    
icd_afib1                    0.344921   0.079118   4.360 1.32e-05 ***
lab_chloride_first          -0.007504   0.004610  -1.628 0.103610    
lab_bnp_flag1                0.022356   0.204183   0.109 0.912818    
lab_troponin_flag1           0.41231

In [10]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),0.2230720,0.08463357,0.5879593
echo1,0.7754224,0.67416127,0.8918934
sofa,1.2203253,1.19408457,1.2471427
vent1,1.4976412,1.21200753,1.8505900
vaso1,1.0509105,0.89435227,1.2348747
sedative1,0.6208378,0.50284857,0.7665121
icd_chf1,1.0587788,0.90382905,1.2402927
icd_afib1,1.4118788,1.20907262,1.6487031
lab_chloride_first,0.9925245,0.98359783,1.0015321
lab_bnp_flag1,1.0226076,0.68534383,1.5258419
